In [17]:
import threading
from hello import hello3
from time import sleep, time

class _Exit(Exception):
    pass

class RunThread(threading.Thread):
    def __init__(self, ip, gpmv=False, timeout=60):
        self.ip = ip
        self.running = False
        self.gpmv = gpmv
        self.timeout = timeout
        super().__init__()
        
    def stop(self):
        self.running = False
        
    def _sleep(self, t):
        end = time() + t
        while time() < end:
            if self.gpmv:
                self.h.gpmv()
            sleep(1)
            if not self.running:
                raise _Exit()
        
    def run(self):
        try:
            self._run()
        except _Exit:
            pass
        print("%s: thread exit" % self.ip)
        
    def _run(self):
        self.running = True
        self.h = h = hello3.HelloApp(self.ip)
        while self.running:
            for t in range(1,10):
                if not self.running:
                    raise _Exit()
                call(h.login)
                sp = h.gpmv()['agitation']['sp']
                call(h.setag, 0, sp)
                print("%s: sleep test for %.1f minute(s)" % (self.ip, t*self.timeout/60))
                self._sleep(self.timeout*t)
                # not wrapped by call() - looking delibrately for 
                # error here
                try:
                    h.setag(0, sp)
                except hello3.ServerCallError as e:
                    print("%s: got error (timeout=%d, gmv=%s): %s" % (self.ip, t, self.gpmv, str(e).splitlines()[0]))
                call(h.logout)
                    
    def __hash__(self):
        return hash(self.ip)
    


In [14]:
threads = {}
def new_thread(ip, mv=False, tm=60):
    t = RunThread(ip, mv, tm)
    threads[ip] = t
    t.start()

def stop_thread(ip):
    threads[ip].stop()
    del threads[ip]
    
def stop_all():
    for t in threads.values():
        t.stop()
    threads.clear()

In [8]:
def call(f, *a, **k):
    
    while True:
        try:
            return f(*a, **k)
        except NotLoggedInError:
            f.__self__.login()
        except hello3.ServerCallError as e:
            print("ServerCallError calling %s for %s: %s" % (f.__name__, f.__self__.ipv4, str(e)))
        except (BadError, IOError, requests.exceptions.ConnectionError) as e:
            print("Error occurred:", e.__class__)   
        time.sleep(1)

In [9]:
new_thread("192.168.1.8", False)
new_thread("192.168.1.14", False)

192.168.1.8: sleep test for 1 minute(s)
192.168.1.14: sleep test for 1 minute(s)
192.168.1.14: got error (timeout=1, gmv=False): No user associated with the set call
192.168.1.8: got error (timeout=1, gmv=False): No user associated with the set call
192.168.1.14: sleep test for 2 minute(s)
192.168.1.8: sleep test for 2 minute(s)
192.168.1.14: got error (timeout=2, gmv=False): No user associated with the set call
192.168.1.8: got error (timeout=2, gmv=False): No user associated with the set call
192.168.1.14: sleep test for 3 minute(s)
192.168.1.8: sleep test for 3 minute(s)
192.168.1.14: got error (timeout=3, gmv=False): No user associated with the set call
192.168.1.8: got error (timeout=3, gmv=False): No user associated with the set call
192.168.1.14: sleep test for 4 minute(s)
192.168.1.8: sleep test for 4 minute(s)
192.168.1.14: got error (timeout=4, gmv=False): No user associated with the set call
192.168.1.8: got error (timeout=4, gmv=False): No user associated with the set call


In [16]:
stop_all()

192.168.1.14: thread exit
192.168.1.8: thread exit


In [11]:
new_thread("192.168.1.8", True)
new_thread("192.168.1.14", True)

192.168.1.8: sleep test for 1 minute(s)
192.168.1.14: sleep test for 1 minute(s)
192.168.1.8: sleep test for 2 minute(s)
192.168.1.14: sleep test for 2 minute(s)
192.168.1.8: sleep test for 3 minute(s)
192.168.1.14: sleep test for 3 minute(s)
192.168.1.8: sleep test for 4 minute(s)
192.168.1.14: sleep test for 4 minute(s)
192.168.1.14: sleep test for 5 minute(s)
192.168.1.8: sleep test for 5 minute(s)


In [15]:
new_thread("192.168.1.8", False, 30)
new_thread("192.168.1.14", False, 30)

192.168.1.8: sleep test for 0 minute(s)
192.168.1.14: sleep test for 0 minute(s)
192.168.1.8: sleep test for 1 minute(s)
192.168.1.14: sleep test for 1 minute(s)
192.168.1.14: got error (timeout=2, gmv=False): No user associated with the set call
192.168.1.8: sleep test for 1 minute(s)
192.168.1.14: sleep test for 1 minute(s)
192.168.1.8: got error (timeout=3, gmv=False): No user associated with the set call
192.168.1.14: got error (timeout=3, gmv=False): No user associated with the set call
192.168.1.8: sleep test for 2 minute(s)
192.168.1.14: sleep test for 2 minute(s)
192.168.1.8: got error (timeout=4, gmv=False): No user associated with the set call
192.168.1.14: got error (timeout=4, gmv=False): No user associated with the set call
192.168.1.8: sleep test for 2 minute(s)
192.168.1.14: sleep test for 2 minute(s)
192.168.1.8: got error (timeout=5, gmv=False): No user associated with the set call
192.168.1.14: got error (timeout=5, gmv=False): No user associated with the set call
192